# CS211: Data Privacy
## In-Class Exercise, Week of 9/26/2022

In [ ]:
# Load the data and libraries
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

adult = pd.read_csv('https://github.com/jnear/cs211-data-privacy/raw/master/homework/adult_with_pii.csv')

## Question 1

For various values of $b$, write code to print out the percent error of summing the ages in the `adult` dataset, 
clipped to each value of $b$.

In [ ]:
bs = range(1, 100, 10)
real_sum = adult['Age'].sum()

# YOUR CODE HERE
raise NotImplementedError()

What value of $b$ is the best?

YOUR ANSWER HERE

## Question 2

For various values of $b$, print the result of a *differentially private* sum of ages, clipped to each value of $b$. Use $\epsilon = 0.1$.

In [ ]:
bs = range(1, 100, 10)
real_sum = adult['Age'].sum()

# YOUR CODE HERE
raise NotImplementedError()

Which value of $b$ is the best now? Does it differ?

YOUR ANSWER HERE

## Question 3

Write an algorithm to *automatically pick the clipping parameter* for the age column. Your algorithm should satisfy differential privacy.

In [ ]:
def pick_b(epsilon):
    # YOUR CODE HERE
    raise NotImplementedError()

pick_b(1.0)

In [ ]:
# TEST CASE for question 3

many_trials = np.mean([pick_b(1.0) for _ in range(100)])
assert many_trials > 70
assert many_trials < 500

## Question 4

What is the privacy cost of your algorithm, and why?

YOUR ANSWER HERE

## Question 5

Write code to generate a *histogram* of education numbers in the `adult` dataset.

In [ ]:
def education_hist():
    # YOUR CODE HERE
    raise NotImplementedError()

education_hist()

In [ ]:
# TEST CASE for question 5
h = education_hist()
assert h['HS-grad'] == 10501
assert h['12th'] == 433
assert h['Doctorate'] == 413

## Question 6

Write code to release a *differentially private histogram* of education numbers.

In [ ]:
def dp_education_hist(epsilon):
    # YOUR CODE HERE
    raise NotImplementedError()

dp_education_hist(1.0)

In [ ]:
# TEST CASE for question 6
h = dp_education_hist(1.0)
assert abs(h['HS-grad'] - 10501) < 100
assert abs(h['Doctorate'] - 413) < 100

## Question 7

What is the total privacy cost of `dp_education_hist`, and why?

YOUR ANSWER HERE

## Question 8

Write code to generate a differentially private *contingency table* for the Education and Sex columns of the `adult` dataset.

In [ ]:
def dp_crosstab_education_sex(epsilon):
    # YOUR CODE HERE
    raise NotImplementedError()

dp_crosstab_education_sex(1.0)

In [ ]:
# TEST CASE for question 8
ct = dp_crosstab_education_sex(1.0)
assert abs(ct['Female']['10th'] - 295) < 100
assert abs(ct['Male']['10th'] - 638) < 100
assert abs(ct['Female']['Bachelors'] - 1619) < 100

assert abs(ct['Female']['10th'] - 295) > 0
assert abs(ct['Male']['10th'] - 638) > 0
assert abs(ct['Female']['Bachelors'] - 1619) > 0

## Question 9

Does parallel composition apply for the contingency table in question 1? Why or why not?

YOUR ANSWER HERE

## Question 10

Does the number of columns used in constructing the contingency table matter for privacy cost? Does it matter for accuracy?

YOUR ANSWER HERE

## Question 11

Implement the Gaussian mechanism for $(\epsilon, \delta)$-differential privacy.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# TEST CASE

results = [gaussian_mech(len(adult[adult['Age'] > 50]), 1, 1.0, 10e-5) for _ in range(100)]
errors = [pct_error(len(adult[adult['Age'] > 50]), r) for r in results]
print('mean error:', np.mean(errors))

assert np.mean(errors) > 0
assert np.mean(errors) < 2

## Question 12

How do the Laplace and Gaussian mechanisms compare in terms of relative error on the query "how many individuals are over 50 years old" with $\epsilon = 1$ and $\delta = 10^{-5}$?

In [ ]:
true_answer = len(adult[adult['Age'] > 50])

laplace_answers = [laplace_mech(true_answer, 1, 1) for _ in range(200)]
gaussian_answers = [gaussian_mech(true_answer, 1, 1, 10e-5) for _ in range(200)]

laplace_error = [pct_error(true_answer, a) for a in laplace_answers]
gaussian_error = [pct_error(true_answer, a) for a in gaussian_answers]

_, bins, _ = plt.hist(gaussian_error, bins=20, label='Gaussian')
plt.hist(laplace_error, bins=bins, label='Laplace', alpha=0.5)
plt.legend();

YOUR ANSWER HERE

## Not a Question - Just for reference

[Reference](https://uvm-plaid.github.io/programming-dp/notebooks/ch6.html#the-gaussian-mechanism)

In [ ]:
epsilon = 1
sensitivity = 1
delta = 1e-5
sigma_squared = 2 * sensitivity**2 * np.log(1.25 / delta) / (epsilon**2)
sigma = np.sqrt(sigma_squared)

def gauss_pdf(x):
    return 1/(sigma*np.sqrt(2*np.pi)) * np.exp(-(1/2)*(x/sigma)**2)

xs = np.linspace(-50, 50, 200)
ys1 = [gauss_pdf(x) for x in xs]
ys2 = [gauss_pdf(x+1) for x in xs]

plt.plot(xs,ys1)
plt.plot(xs,ys2)

# ratio < e^epsilon should hold
print('e^epsilon =', np.exp(epsilon))
ratios = [(x, y1 / y2) for x, y1, y2 in zip(xs, ys1, ys2)]
#ratios

In [ ]:
def laplace_pdf(x):
    return (1/2)*epsilon * np.exp(-np.abs(x)*epsilon)

xs = np.linspace(-50, 50, 200)
ys1 = [laplace_pdf(x) for x in xs]
ys2 = [laplace_pdf(x+1) for x in xs]

plt.plot(xs,ys1)
plt.plot(xs,ys2)

# ratio < e^epsilon should hold
print('e^epsilon =', np.exp(epsilon))
ratios = [(x, y1 / y2) for x, y1, y2 in zip(xs, ys1, ys2)]
#ratios